# BioSTEAM User Guide Tutorial - Beginner's Guide
this runs in Ubuntu 22.04.2 LTS on WSL2, Python 3.10.12\
install BioSTEAM with 'pip install biosteam'\
see https://biosteam.readthedocs.io/en/latest/tutorial/index.html

## 6. Convergence
see https://biosteam.readthedocs.io/en/latest/tutorial/Convergence.html

Here is a list of defaults recycle systems use:

In [7]:
from biosteam import System
System.default_maxiter # -> 200
System.default_molar_tolerance # -> 1. [kmol/hr] for each component
System.default_relative_molar_tolerance # -> 0.01
System.default_temperature_tolerance # -> 0.10 [K]
System.default_relative_temperature_tolerance # -> 0.001
System.default_method # -> 'Aitken'
# Changing any of these would change the default for
# all new recycle systems you create.



'Aitken'

For a recycle system to converge, the difference between molar flow rate of the final and previous iteration must below either the relative or absolute molar tolerance (not both). Because the default molar tolarances are as wide as 1% or 0.01 kmol/hr, resimulating a system may give slightly different results in some cases. In the following example, we create a simple recycle loop consisting of a flash, a mixer, and a splitter:

In [1]:
import biosteam as bst
bst.nbtutorial() # Light-mode html diagrams and filter warnings
bst.settings.set_thermo(['Water'])
feed = bst.Stream('feed', Water=1)
recycle = bst.Stream('recycle')
liquid_product = bst.Stream('liquid_product')
mixer = bst.Mixer('mixer', [feed, recycle])
flash = bst.Flash('flash', mixer-0, ['vapor_product', ''], Q=feed.Hvap, P=101325)
splitter = bst.Splitter('splitter', flash.liquid, outs=[recycle, liquid_product], split=0.1) # flash.liquid is flash.outs[1]
sys = bst.main_flowsheet.create_system('sys')
sys.simulate()
sys.diagram()
sys.show()

System: sys
Highest convergence error among components in recycle
stream splitter-0 after 2 loops:
- flow rate   5.71e-04 kmol/hr (9.1%)
- temperature 0.00e+00 K (0%)
ins...
[0] feed  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water  1
outs...
[0] vapor_product  
    phase: 'g', T: 373.12 K, P: 101325 Pa
    flow (kmol/hr): Water  0.943
[1] liquid_product  
    phase: 'l', T: 373.12 K, P: 101325 Pa
    flow (kmol/hr): Water  0.0566


### 6.1.2 Convergence methods

It may be possible that a system may be more efficient to converge using another solver. Here is a list of solvers available in BioSTEAM:

In [2]:
list(bst.System.available_methods)

# Aitken, Wegstein, and fixed-point methods are in-house solvers from BioSTEAM. The other methods are imported from scipy.

['aitken',
 'wegstein',
 'fixedpoint',
 'anderson',
 'diagbroyden',
 'excitingmixing',
 'linearmixing',
 'broyden1',
 'broyden2']

### 6.1.3. Importance of thermodynamic property package

The choice of thermodynamic property package may have a large impact in the results, regardless of convergence. For example, in the liquefaction of nitrogen, it is important to account for excess thermodynamic energies due to high pressures.

In [3]:
import biosteam as bst
bst.main_flowsheet.clear()

# Setup thermodynamic backend
N2 = bst.Chemical('N2')
bst.settings.set_thermo([N2])

# Heat integration between inlet and recycle
inlet = bst.Stream("inlet", N2=2.75, units="kg/s", T=6.85+273.15, P=200e5, phase="g")
flash_gas = bst.Stream("flash_gas")
regenerator = bst.units.HXprocess("regenerator", ins=(inlet, flash_gas), outs=("throttle_in", 'out'), dT=9.55)
throttle_in = regenerator.outs[0]

# Throttling
valve = bst.units.IsenthalpicValve("expansion", ins=throttle_in, outs="valve_out", P=1e5, vle=True)
valve_out = valve.outs[0]

# Flash drum
flash = bst.units.Flash("flash", ins=valve_out, outs=(flash_gas, "flash_liquid"), Q=0, P=1e5)

# Connect flash gas to regenerator
regenerator.ins[1] = flash.vapor

sys = bst.main_flowsheet.create_system('N2_liquefaction')
sys.set_tolerance(mol=1e-9, rmol=1e-9, T=1e-5, rT=1e-5)
bst.settings.mixture.include_excess_energies = False
sys.simulate()
sys.diagram()
print('Results without excess free energies')
print('------------------------------------')
sys.show()
print()
print('Results with excess free energies')
print('------------------------------------')
sys.reset_cache() # Resets cached chemical/mixture properties
bst.settings.mixture.include_excess_energies = True
sys.empty_recycles()
sys.simulate()
sys.show()

Results without excess free energies
------------------------------------
System: N2_liquefaction
Highest convergence error among components in recycle
stream flash-0 after 2 loops:
- flow rate   0.00e+00 kmol/hr (0%)
- temperature 0.00e+00 K (0%)
ins...
[0] inlet  
    phase: 'g', T: 280 K, P: 2e+07 Pa
    flow (kmol/hr): N2  353
outs...
[0] out  
    phase: 'g', T: 280 K, P: 100000 Pa
    flow (kmol/hr): N2  353
[1] flash_liquid  
    phase: 'l', T: 280 K, P: 100000 Pa
    flow: 0

Results with excess free energies
------------------------------------
System: N2_liquefaction
Highest convergence error among components in recycle
stream flash-0 after 11 loops:
- flow rate   1.71e-13 kmol/hr (0%)
- temperature 0.00e+00 K (0%)
ins...
[0] inlet  
    phase: 'g', T: 280 K, P: 2e+07 Pa
    flow (kmol/hr): N2  353
outs...
[0] out  
    phase: 'g', T: 270.45 K, P: 100000 Pa
    flow (kmol/hr): N2  325
[1] flash_liquid  
    phase: 'l', T: 77.244 K, P: 100000 Pa
    flow (kmol/hr): N2  28.2


At times, a process specification might involve having to rerun upstream unit operations. In such cases, it may be easier to set the system to run more than once than to code-in the simulation of upstream units. For demonstration purposes, let’s specify a mixer’s product flow rate of water to be constant by varying one of the feeds:

In [4]:
bst.main_flowsheet.clear()
bst.settings.set_thermo(['Water'])
feed_a = bst.Stream('feed_a')
heat_exchanger = bst.HXutility('heat_exchanger', feed_a, T=320, rigorous=True)
feed_b = bst.Stream('feed_b', Water=1e3)
mixer = bst.Mixer('mixer', [heat_exchanger-0, feed_b])
mixer_product_flow_specification = 1e6 # kmol/hr
@mixer.add_specification
def adjust_fresh_flow():
    # Updates the feed, but doesn't run the upstream heat exchanger.
    # A good solution would be to also run the heat exchanger, but here
    # we do not for demonstration purposes
    feed_a.imol['Water'] = mixer_product_flow_specification - feed_b.imol['Water']
    mixer.run()

sys = bst.main_flowsheet.create_system('sys')
sys.simulate()
heat_exchanger.show()

HXutility: heat_exchanger
ins...
[0] feed_a  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water  9.99e+05
outs...
[0] s1  to  Mixer-mixer
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


Although we changed the feed upstream of the heat exchanger, notice how we will need to rerun the heat exchanger. We can do this by setting the number of times a system runs using N_runs:

In [6]:
#In general, depending on N_runs is not recommended, but it may be useful when dealing with complex process specifications.
sys.N_runs = 2 # Run twice in simulate method
sys.empty_recycles()
sys.simulate()
heat_exchanger.show()

HXutility: heat_exchanger
ins...
[0] feed_a  
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water  9.99e+05
outs...
[0] s1  to  Mixer-mixer
    phases: ('g', 'l'), T: 320 K, P: 101325 Pa
    flow (kmol/hr): (l) Water  9.99e+05


In [ ]:
## 6.2. Phase equilibrium and energy balance
### 6.2.1. Adjusting convergence criteria